# Which GPU to use

In [1]:
multiGPU = False
whichGPU = 0
 
# Select which GPU to use
if(multiGPU):
    from keras.utils.training_utils import multi_gpu_model
else:
    import os
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    # The GPU id to use, usually either "0" or "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(whichGPU)
    
# # Do other imports now...

# Load all the functions

In [2]:
%run -i 'arena.py'

Using TensorFlow backend.


# General Parameters

In [3]:
import math

# What data to use
tableBase = '4PpKk'
convertStates = False

# Interactive (just in general if one is asked for confirmations, set to False if on autopilot over night f.x.)
askForConfirmation = False

# NN parameters
filters = [16,32,32,64,128,128,128]
filterShape = [2,2,2,2,2,2,2]
batch_size = 256
optimizer = 'Adadelta'
useBatchNorm = False
num_classes = 3
input_shape = (4,8,8)

### DON'T MODIFY BELOW ###
# Generate dataset variables
fileName = tableBase + '.hdf5'
dataSetName = tableBase + '_onlyLegal'
if not convertStates: 
    dataSetName = tableBase + '_onlyLegal_fullStates'
dataSetWdlName = tableBase + '_Wdl_onlyLegal_3Values'

# Number of Pieces
nPi =  int(dataSetName[0])
nPa = nPi - 2
nWPa = math.ceil(nPa/2)

# Experiment 1
Bengio methood
3n4 no freeze

### Exp 1 Paramters

In [4]:
# 3n4 no freeze 
%run -i 'arena.py'

# Parameters
sourceNet = '103' # trained on 3pc from scratch
# sourceNet = '107' # trained on 4pc from scratch
freeze = False
resSaveFile = '3n4nofreeze'
epochs = 1
averageOver = 1
expDescr = "Bengio 3n4 - freeze = {} - average over {} runs".format(str(freeze), averageOver)

saveEveryRun = False # save stuff in results dir
saveWeightsCheckpoints = False # save chkp in results dit
saveTensorboardLogs = False # save logs in ./logs dir
resID = '---NORESID---' # used when not saving data, but fitModel() still needs a resID

fractionOfDataToUse = 0.01
plotDuringTraining = False
loadWeights = False 
askForConfirmation = False
saveDir = 'bengioResults'

resSaveFile = resSaveFile + '-{}runAverage'.format(averageOver)
resSaveFileFullPath = saveDir + '/' + str(resSaveFile) + '.pkl' 

### Create model and load data

In [5]:
# prepare save file
if not os.path.exists(resSaveFileFullPath):
    print("Save file doesn't exists, creating...\n")
    save_obj(saveDir, resSaveFile, [])
else:
    print("Save file exists...\n")


# load data
X_train, X_test, y_train, y_test = loadData()

# create model
model, nnStr = createModel()
layersCount = len(model.layers)

# load old results
results = load_obj(saveDir, resSaveFile)

# initialize variables wrt old results
startTrainingAtLayer = len(results)
print("\nStarting/restarting TL at {} transfered layers".format(startTrainingAtLayer))

Save file doesn't exists, creating...

X_train shape: (49821, 4, 8, 8)
y_train shape: (49821, 1)
X_test shape: (24539, 4, 8, 8)
y_test shape: (24539, 1)
49821 train samples
24539 test samples
Done loading dataset
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 7, 7)          272       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 6, 32)         928       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 5, 32)         4128      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 4, 64)         8256      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 3, 128)        32896     
_____________________________________________________________

### Train

In [6]:
%run -i 'arena.py'

for copyFirstNLayers in range(startTrainingAtLayer, layersCount):
    print('\n\n')
    print('==========================================================================================')
    print('=                                                                                        =')
    print('=                    Currently transfering first {} layers, out of {}                      ='.format(copyFirstNLayers, layersCount - 1))
    print('=                                                                                        =')
    print('==========================================================================================')
    print()
    
    if copyFirstNLayers == layersCount - 1:
        copyFirstNLayers += 1
        
    accumulatedScore = 0
    for a in range(averageOver):
        # save current averagePosition to tmp file
        with open(saveDir + '/' + str(resSaveFile) + '_currentPosition.txt','w') as file:
            file.write('Layers Transfered: {}\nInner avg loop position: {} out of {}'.format(copyFirstNLayers, a, averageOver)) 
        
        model = loadNFirstLayers(model, sourceNet, copyFirstNLayers , freeze)

        # Prepare save dir
        if saveEveryRun:
            resID = genNextResultsDir(model)

        # train
        fitHistory, logDir = trainModel(resID, model, saveWeightsCheckpoints, saveTensorboardLogs)

        # score and save results
        score = calcScore(model)
        if saveEveryRun:
            saveTrainResults(resID, model, logDir, score, copyFirstNLayers)

        # update Return
        accumulatedScore += score[1]
        
    # append averaged results for one set of layers
    results.append(accumulatedScore/averageOver)

    # save old results to checkpoints dir
    dateTime = time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime())
    src = saveDir + '/' + str(resSaveFile) + '.txt'
    dest = saveDir + '/checkpoints/' + str(resSaveFile) + dateTime + '.txt'
    if os.path.exists(src):
        shutil.move(src, dest)

    # save results 
    save_obj(saveDir, resSaveFile, results)
    with open(saveDir + '/' + str(resSaveFile) + '.txt','w') as file:
        file.write(str(results))

    # to load:
    # results = load_obj('temp','3n4.txt')
print('\n Final Results: {}'.format(results))




=                                                                                        =
=                    Currently transfering first 0 layers, out of 8                      =
=                                                                                        =

Loading first 0 layers from results Results/103/weights.hdf5, 
- Resetting layer nr 1: <keras.layers.convolutional.Conv2D object at 0x7ff234778390>
- Resetting layer nr 2: <keras.layers.convolutional.Conv2D object at 0x7ff2347789e8>
- Resetting layer nr 3: <keras.layers.convolutional.Conv2D object at 0x7ff23470cc88>
- Resetting layer nr 4: <keras.layers.convolutional.Conv2D object at 0x7ff23471f4e0>
- Resetting layer nr 5: <keras.layers.convolutional.Conv2D object at 0x7ff234736908>
- Resetting layer nr 6: <keras.layers.convolutional.Conv2D object at 0x7ff2346e67f0>
- Resetting layer nr 7: <keras.layers.convolutional.Conv2D object at 0x7ff2346ff550>
- Skipping layer nr 8: <keras.layers.core.Flatten object at 0x7ff

49821/49821 [==============================] - 2s 34us/step - loss: 0.7138 - acc: 0.6824 - val_loss: 0.5376 - val_acc: 0.7759
Training done
Calculating score
24539/24539 [==============================] - 1s 45us/step
(49821, 4, 8, 8)
Evaluated test loss: 0.5376178416535802
Evaluated test accuracy: 0.7759077387235194



=                                                                                        =
=                    Currently transfering first 6 layers, out of 8                      =
=                                                                                        =

Loading first 6 layers from results Results/103/weights.hdf5, 
- Resetting layer nr 7: <keras.layers.convolutional.Conv2D object at 0x7ff2346ff550>
- Skipping layer nr 8: <keras.layers.core.Flatten object at 0x7ff234697f98>
- Resetting layer nr 9: <keras.layers.core.Dense object at 0x7ff234697e10>
Train on 49821 samples, validate on 24539 samples
Epoch 1/1
49821/49821 [==============================] 